In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import itertools
import math

%matplotlib inline

# pd.set_option("display.max_rows", 10)

np.random.seed(2)

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding


Using TensorFlow backend.


In [148]:
# Load the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train_X = train.iloc[:,1:]
train_y = train.iloc[:,-1]

train_X.shape

(1460, 80)

In [149]:
set_1_num = train_X[['OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'SalePrice']]
set_1_cat = train_X[['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'SaleCondition', 'SaleType']]
set_1_cat_test = test[['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'SaleCondition', 'SaleType']]

set_2 = train_X[['BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'SalePrice']]

set_3 = train_X[['GrLivArea', 'LotArea', 'GarageArea', 'OpenPorchSF', 'SalePrice']]

set_1_cat[set_1_cat['Electrical'] == np.nan]

set_1_cat['Electrical'] = set_1_cat['Electrical'].fillna('Unk')
set_1_cat.isnull().sum()


/home/daniele/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


MSZoning         0
Utilities        0
LotConfig        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
Exterior1st      0
Exterior2nd      0
ExterCond        0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
SaleCondition    0
SaleType         0
dtype: int64

In [151]:

labels = np.array([])
for c_name in set_1_cat.columns.values:
    labels = np.append(labels, set_1_cat[c_name].unique())
for c_name in set_1_cat_test.columns.values:
    labels = np.append(labels, set_1_cat_test[c_name].unique())
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(labels.tolist())

for c_name in set_1_cat.columns.values:
    set_1_cat[c_name] = label_encoder.transform(set_1_cat[c_name])
    
train_X = pd.concat([set_1_num, set_1_cat, set_2, set_3], axis=1)

train_X = train_X.drop('SalePrice', axis=1)

print(train_X.shape)

train_X.sample(5)

/home/daniele/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


(1460, 32)


OverallQual  OverallCond  YearBuilt  YearRemodAdd  MSZoning  Utilities  \
467             5            7       1942          1995        88         10   
1011            5            5       1965          1965        88         10   
895             6            5       1963          1963        88         10   
1426            7            5       1994          1994        88         10   
635             6            7       1914          1995        87         10   

      LotConfig  Neighborhood  Condition1  Condition2     ...       SaleType  \
467          62            69          12          75     ...            112   
1011         62            38          75          75     ...            112   
895          62            69          75          75     ...            112   
1426         62            73          75          75     ...            112   
635          62            97          45          75     ...            112   

      BsmtFullBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  GrLivArea  \
467              0         1         1             3             1       1644   
1011             0         2         0             4             2       1664   
895              1         1         1             4             1       1796   
1426             1         2         1             3             1       2127   
635              0         2         0             8             2       3395   

      LotArea  GarageArea  OpenPorchSF  
467      9480         312            0  
1011     9825           0            0  
895      7056         483           50  
1426    10944         525          132  
635     10896           0          110  

[5 rows x 32 columns]

In [152]:
reg = LinearRegression().fit(train_X, train_y)

predict = reg.predict(train_X).round()

print('RMSE = %.6f' % math.sqrt(mean_squared_error(predict, train_y)))


RMSE = 35133.027430


In [153]:
set_1_num = test[['OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd']]
set_1_cat = test[['MSZoning', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'SaleCondition', 'SaleType']]

set_2 = test[['BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', ]]

set_3 = test[['GrLivArea', 'LotArea', 'GarageArea', 'OpenPorchSF', ]]

set_1_cat[set_1_cat['Electrical'] == np.nan]

set_1_cat['Electrical'] = set_1_cat['Electrical'].fillna('Unk')
set_1_cat.isnull().sum()


labels = np.array([])

for c_name in set_1_cat.columns.values:
    set_1_cat[c_name] = label_encoder.transform(set_1_cat[c_name])
    
test = pd.concat([set_1_num, set_1_cat, set_2, set_3], axis=1)


test_predict = reg.predict(test).round()

test_predict = pd.DataFrame(test_predict)

predict.to_csv('submission.csv')


/home/daniele/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


ValueError: y contains previously unseen labels: nan